In [ ]:
import numpy as np
import pandas as pdimport matplotlib.pyplot as plt


In [ ]:
diam = 5; #diameter of the mug, cm

del_l = 0.05; #cell size, cm

N_d = int(diam/del_l); #number of diameter cells

#cell centre of sclalar CV
X = np.arange(del_l/2, diam, del_l);
Y = np.arange(del_l/2, diam, del_l);

mug_X, mug_Y = np.meshgrid(X, Y);

#ATTENTION THAT THE MUG MATRIX IS LARGER THAN THE ACTUAL XY GRID
mug = np.zeros((N_d + 2, N_d + 2)); 
#create the domain matrix #modification 1 in order to create staggered arrangement
#and have a matrix capable of accomodating BC

print(np.shape(X))
print(mug_X)
print(mug_Y)

In [ ]:
def circle(x, y, c_x, c_y):
    r = (x - c_x)**2 + (y - c_y)**2;
    return r

In [ ]:
centre_x = diam/2; 
centre_y = diam/2; 
radius = diam/2; #location centre and radius mug

for a_y in range(0, N_d):
    for a_x in range(0, N_d):
        if circle(mug_X[a_y, a_x], mug_Y[a_y, a_x], centre_x, centre_y) < radius**2:
            mug[a_y + 1, a_x + 1] = 1; #adapted to the new mug matrix arrangement
        else:
            mug[a_y + 1, a_x  + 1] = 0; #not necessary

In [ ]:
domain_shape = pd.DataFrame(mug);
domain_shape.to_excel("mug_shape.xlsx")

#if you check the excel document you will see that due the discretisation there is a thickness 1-cell layer of zeros
#surrounding the 1s corresponding to the mug interior

In [ ]:
#define the velocity field
#defined at the cell centre
#useful for representation

omega = 0.05; #rad/s

U = -(mug_Y - centre_y*np.ones((N_d, N_d)))*omega;
V = (mug_X - centre_x*np.ones((N_d, N_d)))*omega;

for a_y in range(0, N_d):
    for a_x in range(0, N_d):
        if mug[a_y + 1, a_x + 1] == 0: #adapted to the new mug matrix arrangement
            U[a_y, a_x] = 0;
            V[a_y, a_x] = 0;
        else:
            continue

U_velocity_field = pd.DataFrame(U);
V_velocity_field = pd.DataFrame(V);

U_velocity_field.to_excel("U_vel_field.xlsx");
V_velocity_field.to_excel("V_vel_field.xlsx");

In [ ]:
cells_per_row = sum(mug);
print(cells_per_row);

In [ ]:
a = sum(cells_per_row[0:N_d + 2])
print(a)

In [ ]:
#pre-allocate memory for the data structure
#needed for maximum efficiency

size_data_structure = int(sum(cells_per_row)*5);

sol_mat = np.zeros((size_data_structure,3));

for N in range(0, size_data_structure):
    a = N*5;
    b = a + 5;
    sol_mat[a:b, 0] = N; #count only the cells composing the mug
    
    
#consider all the possible cases
#same structure of registers of 5 cells

# S = 0; W = 1; P = 2; E = 3; N = 4;

#DEFINE THE GEOMETY OF THE CIRCULAR DOMAIN/MUG
#all cells are surrounded by at most 2 zero cells
for a_y in range(1, N_d + 1):
    for a_x in range(1, N_d + 1):
        
        #creation of the gross structure
        if mug[a_y, a_x] == 0:
            continue
        elif mug[a_y, a_x] == 1:
            num_0_left = (N_d - cells_per_row[a_y])/2;
            cell_data_N = sum(cells_per_row[0 : a_y]) + (a_x - 1) - num_0_left;
            
            num_sol_mat = int(cell_data_N*5);
            S = num_sol_mat;
            W = num_sol_mat + 1;
            P = num_sol_mat + 2;
            E = num_sol_mat + 3;
            N = num_sol_mat + 4;
            
            S_cell = sum(cells_per_row[0 : (a_y - 1)]) + (a_x - 1) - (N_d - cells_per_row[a_y - 1])/2;
            W_cell = cell_data_N -1;
            E_cell = cell_data_N +1;
            N_cell = sum(cells_per_row[0 : (a_y + 1)]) + (a_x - 1) - (N_d - cells_per_row[a_y + 1])/2;
            
            sol_mat[S, 1] = int(S_cell);
            sol_mat[W, 1] = int(W_cell);
            sol_mat[P, 1] = int(cell_data_N);
            sol_mat[E, 1] = int(E_cell);
            sol_mat[N, 1] = int(N_cell);
        
#boundary refinement
for a_y in range(1, N_d + 1):
    for a_x in range(1, N_d + 1):
        if mug[a_y, a_x] == 0:
            continue
        elif mug[a_y, a_x] == 1:
            num_0_left_BC = (N_d - cells_per_row[a_y])/2;
            cell_data_N_BC = sum(cells_per_row[0 : a_y]) + (a_x - 1) - num_0_left_BC;
        
            num_sol_mat_BC = int(cell_data_N_BC*5);
            S_BC = num_sol_mat_BC;
            W_BC = num_sol_mat_BC + 1;
            P_BC = num_sol_mat_BC + 2;
            E_BC = num_sol_mat_BC + 3;
            N_BC = num_sol_mat_BC + 4;
            
            if mug[a_y, a_x - 1] == 0:
                sol_mat[W_BC, 1] = 0;
            if mug[a_y, a_x + 1] ==0:
                sol_mat[E_BC, 1] = 0;
            if mug[a_y - 1, a_x] == 0:
                sol_mat[S_BC, 1] = 0;
            if mug[a_y + 1, a_x] == 0:
                sol_mat[N_BC, 1] = 0;
        
#checking the first few cells I see that the data ctructure has been created as desired

In [ ]:
#define the physical constants and the BC

#all refer to a liquid of low thermal conductivity with the same density of water
rho = 1; #g/cm^3
heat_c = 4.182; #J/gK at 25 deg C
A = del_l**2; #area cm^2
kappa = 0.1; #W/cm*K at 20 deg C (for homogeneity)

#diffusive conductance
D = (kappa/del_l);

In [ ]:
#redifine velocity field for staggered arrangement
#calculate the number of velocity cells needed

velocity_cells_per_row_U = (cells_per_row[1 : N_d + 1] - 1);
velocity_cells_per_row_V = cells_per_row[1 : N_d + 1];
print(velocity_cells_per_row_U);
print(velocity_cells_per_row_V)

num_U_star = int(sum(velocity_cells_per_row_U)); #West-East velocity

num_V_star = 0;
for t in range(0, N_d - 1):
    if velocity_cells_per_row_V[t] <= velocity_cells_per_row_V[t + 1]:
        num_V_star = num_V_star + velocity_cells_per_row_V[t];
    else:
        num_V_star = num_V_star + velocity_cells_per_row_V[t + 1];

num_V_star = int(num_V_star)
num_U_star = int(num_U_star)

In [ ]:
#calculate the velocities in the staggered arrangement

#allocate memory for velocity vectors
U_star = np.zeros((1, num_U_star));
V_star = np.zeros((1, num_V_star));

#define staggered gird for velocity calculation, check notes
X_vel = np.arange(del_l, diam + del_l, del_l);
Y_vel = np.arange(del_l, diam + del_l, del_l);

print(X_vel)
print(np.shape(X_vel))

mug_X_vel, mug_Y_vel = np.meshgrid(X_vel, Y_vel);

#notice the alternative way of counting the cells that enter the velocity
location_U = 0
#calculation of u velocity
for a_y in range(0, N_d):
    for a_x in range(0, N_d):
        if mug[a_y + 1, a_x + 1] == 0:
            continue
        elif (mug[a_y + 1, a_x + 1] == 1 and mug[a_y + 1, a_x + 2] != 0): #don't count the ones that have a 0 to their right, boundary has 0 vel
            U_star[0, location_U] = -(Y_vel[a_y] - centre_y)*omega;
            location_U = location_U + 1;
        else:
            continue

print(U_star)

location_V = 0;
#calculation of v velocity
for a_y in range(0, N_d):
    for a_x in range(0, N_d):
        if mug[a_y + 1, a_x + 1] == 0:
            continue
        elif (mug[a_y + 1, a_x + 1] == 1 and mug[a_y + 2, a_x + 1] == 0):
            continue
        else:
            V_star[0, location_V] = (X_vel[a_x] - centre_x)*omega;
            location_V = location_V +1;

print(V_star)

#we have the staggered velocities ready

In [ ]:
#definition of flows

F_u = U_star*rho*heat_c;
F_v = V_star*rho*heat_c;

print(F_u)
print(F_v)

In [ ]:
#definition of boundaries, scalar temperature boundary
T_down = 11; #in
T_up = 1; #out

#ATTENTION, EQUATING VARIABLES IN PHYTON IS EQUIVALENT TO SAY THAT THEIR NAMES/ADDRESSES IN MEMORY ARE THE SAME
#IF YOU NEED DO CREATE A !!!NEW VARIABLE!!! STARTING FROM AN OLD ONE YOU MUST EQUATE ELEMENTWISE
mug_BC = np.zeros((N_d + 2, N_d + 2));

for m in range(0, N_d + 1):
    for n in range(0, N_d + 1):
        mug_BC[m, n] = mug[m, n];

#bottom half at temperature T_down
#given that the enlarged mug matrix has size N_d + 2

#must pay attention to the direction of the heat flow
for a_y in range(0, int(N_d/2) + 1):
    for a_x in range(0, N_d + 2):
        if mug[a_y, a_x] == 1:
            continue
        elif  mug[a_y, a_x] == 0:
            if a_x == 0 and mug[a_y, a_x + 1] == 1:
                mug_BC[a_y, a_x] = T_down; #W
            if a_x == (N_d + 1) and mug[a_y, a_x - 1] == 1:
                 mug_BC[a_y, a_x] = T_down; #E
            elif a_x != 0 and a_x != (N_d + 1):
                if (mug[a_y, a_x] == 0 and mug[a_y + 1, a_x] == 1):
                    mug_BC[a_y, a_x] = T_down; #S
                elif (mug[a_y, a_x] == 0 and mug[a_y, a_x + 1] == 1):
                    mug_BC[a_y, a_x] = T_down; #W
                elif (mug[a_y, a_x] == 0 and mug[a_y, a_x - 1] == 1):
                    mug_BC[a_y, a_x] = T_down; #E

# top half temperature

for a_y in range(int(N_d/2) + 1, N_d + 2):
    for a_x in range(0, N_d + 2):
        if mug[a_y, a_x] == 1:
            continue
        elif  mug[a_y, a_x] == 0:
            if a_x == 0 and mug[a_y, a_x + 1] == 1:
                mug_BC[a_y, a_x] = T_up; #W
            if a_x == (N_d + 1) and mug[a_y, a_x - 1] == 1:
                 mug_BC[a_y, a_x] = T_up; #E
            elif a_x != 0 and a_x != (N_d + 1):
                if (mug[a_y, a_x] == 0 and mug[a_y - 1, a_x] == 1):
                    mug_BC[a_y, a_x] = T_up; #N
                elif (mug[a_y, a_x] == 0 and mug[a_y, a_x + 1] == 1):
                    mug_BC[a_y, a_x] = T_up; #W
                elif (mug[a_y, a_x] == 0 and mug[a_y, a_x - 1] == 1):
                    mug_BC[a_y, a_x] = T_up; #W

In [ ]:
#FULLL UPWIND SCHEME FOR CIRCULAR DOMAIN
#VALID IN GENERAL, YOU ONLY NEED TO MODIFY THE DATASTRUCTURE, I.E. WHERE YOU PICK THE CELLS

#IMPORVED WITH LOGICAL STATEMENTS FOR PATHOLOGICAL CASES OF ZERO VELOCITY OR CHANGING VELOCITY, VELOCITY = 0
#must account for all the cases in whcih the velocity is 0 in contiguous cells
# only in 1, in both in a direction and in all four (centre of the flow for a vortex)

#BOUNDARY CELLS MUST BE ADJUSTED DEPENDING ON THE KIND OF FLOW PROBLEM
#THE CASE BY CASE STRUCTURE FOR THE BOUNDARY IS HELPFUL IN THIS SENSE

#define a vector of the number of cell difference between adjacent rows, needed for the bottom half
row_diff_vect = np.zeros((1, int(N_d/2)));
for i in range(0, int(N_d/2)):
    diff = cells_per_row[i + 1] - cells_per_row[i];
    row_diff_vect[0, i] = diff;
#half of it    
half_row_diff_vect = row_diff_vect/2;
#we are going to use these two vectors to compute the locations of the V velocities wrt the scalar CV

number_of_cell = 0;

for a_y in range(0, N_d):
    for a_x in range(0, N_d):
        
        num_0_left = (N_d - cells_per_row[a_y + 1])/2;
        location_cell = int(sum(cells_per_row[1 : (a_y + 1)]) + a_x - num_0_left)
        #the cell number is accounted for in this more efficient way (count the cells inside the 1s domain)
        
        register = int(location_cell*5);
        S = int(register);
        W = register + 1;
        P = register + 2;
        E = register + 3;
        N = register + 4;
            
        #define the velocity vectors that will surround the cells inside the itaration algorithm
        #first get rid of the BC accounting for the individual case
        #then we have the general algorithm for the remaining cells
        
        #ATTENTION THAT THE DOMAIN IS UPSIDE-DOWN
        if mug[a_y + 1, a_x + 1] == 0:
            continue
        #Boundaries
        elif mug[a_y + 1, a_x + 1] == 1:
            
            number_of_cell = number_of_cell + 1;
            
            Pe_u = 0;
            Pe_v = 0;
            
            ###########################################################################################################
            #define the velocity cells surrounding the scalar CV
            W_cell_U = int(sol_mat[W, 1] - a_y);
            E_cell_U = int(sol_mat[E, 1] - a_y - 1);
            
            #for the S and N velocity cells we need a if-statement
            if a_y < N_d/2:
                N_cell_V = int(sol_mat[P, 1]);
                S_cell_V = int(sol_mat[S, 1]);
            elif a_y == N_d/2: #exception
                t = int(a_y - N_d/2)
                N_cell_V = int(sol_mat[P, 1] - sum(row_diff_vect[0, 0 : t]) - half_row_diff_vect[0, t]);
                S_cell_V = int(sol_mat[S, 1]);
            elif a_y > N_d/2:
                t = int(a_y - N_d/2)
                N_cell_V = int(sol_mat[P, 1] - sum(row_diff_vect[0, 0 : t]) - half_row_diff_vect[0, t]);
                S_cell_V = int(sol_mat[S, 1] - sum(row_diff_vect[0, 0 : t - 1]) - half_row_diff_vect[0, t - 1])
            
            alternative_U = (F_u[0, E_cell_U]+F_u[0, W_cell_U])/2;
            alternative_V = (F_v[0, N_cell_V]+F_v[0, S_cell_V])/2;
            ###########################################################################################################
            
            if (mug[a_y + 1, a_x] == 0 and mug[a_y, a_x + 1] == 0): #S, W corner
                sol_mat[S, 2] = 0;
                sol_mat[W, 2] = 0;
                
                if F_u[0, E_cell_U] >= 0:
                    a_E = - D;
                elif F_u[0, E_cell_U] < 0:
                    a_E = F_u[0, E_cell_U] - D;
                if F_v[0, N_cell_V] >= 0:
                    a_N = -D;
                elif F_v[0, N_cell_V] < 0:
                    a_N = F_v[0, N_cell_V] - D;
                
                sol_mat[P, 2] = F_u[0, E_cell_U] + F_v[0, N_cell_V] - a_E - a_N + 4*D;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
            elif (mug[a_y + 1, a_x + 2] == 0 and mug[a_y, a_x + 1] == 0): #S, E corner
                sol_mat[S, 2] = 0;
                sol_mat[E, 2] = 0;
                
                if F_u[0, W_cell_U] > 0:
                    a_W = -F_u[0, W_cell_U] - D;
                elif F_u[0, W_cell_U] <= 0:
                    a_W = - D;
                if F_v[0, N_cell_V] >= 0:
                    a_N = -D;
                elif F_v[0, N_cell_V] < 0:
                    a_N = F_v[0, N_cell_V] - D;
                
                sol_mat[W, 2] = a_W
                sol_mat[N, 2] = a_N
                sol_mat[P, 2] = - F_u[0, W_cell_U] + F_v[0, N_cell_V] - a_W - a_N + 4*D;
            elif (mug[a_y + 1, a_x] == 0 and mug[a_y + 2, a_x + 1] == 0): #N, W corner    
                sol_mat[W, 2] = 0;
                sol_mat[N, 2] = 0;
                
                if F_u[0, E_cell_U] >= 0:
                    a_E = - D;
                elif F_u[0, E_cell_U] < 0:
                    a_E = F_u[0, E_cell_U] - D;
                if F_v[0, S_cell_V] > 0:
                    a_S = - F_v[0, S_cell_V] - D;
                elif F_v[0, S_cell_V] <= 0:
                    a_S = - D;
                
                sol_mat[S, 2] = a_S
                sol_mat[P, 2] = F_u[0, E_cell_U] - F_v[0, S_cell_V] - a_E - a_S  + 4*D;
                sol_mat[E, 2] = a_E;
                
            elif (mug[a_y + 1, a_x + 2] == 0 and mug[a_y + 2, a_x + 1] == 0): #N, E corner
                sol_mat[E, 2] = 0;
                sol_mat[N, 2] = 0;
                
                if F_u[0, W_cell_U] > 0:
                    a_W = -F_u[0, W_cell_U] - D;
                elif F_u[0, W_cell_U] <= 0:
                    a_W = - D;
                if F_v[0, S_cell_V] > 0:
                    a_S = - F_v[0, S_cell_V] - D;
                elif F_v[0, S_cell_V] <= 0:
                    a_S = - D;
                
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W
                sol_mat[P, 2] = - F_u[0, W_cell_U]- F_v[0, S_cell_V] - a_S - a_W + 4*D;
                
            elif mug[a_y + 1, a_x] == 0: #W wall
                sol_mat[W, 2] = 0;
                
                if F_u[0, E_cell_U] >= 0:
                    a_E = - D;
                elif F_u[0, E_cell_U] < 0:
                    a_E = F_u[0, E_cell_U] - D;
                if F_v[0, N_cell_V] >= 0 or alternative_V >= 0:
                    a_S = -F_v[0, S_cell_V] - D;
                    a_N = -D
                elif F_v[0, N_cell_V] < 0 or alternative_V < 0:
                    a_S = - D;
                    a_N = F_v[0, N_cell_V] - D;
                
                sol_mat[S, 2] = a_S
                sol_mat[E, 2] = a_E
                sol_mat[N, 2] = a_N
                sol_mat[P, 2] = (F_u[0, E_cell_U] + F_v[0, N_cell_V] - F_v[0, S_cell_V]) - a_E - a_N - a_S + 2*D;
            elif mug[a_y + 1, a_x + 2] == 0: #E wall
                sol_mat[E, 2] = 0;
                
                if F_u[0, W_cell_U] > 0:
                    a_W = -F_u[0, W_cell_U] - D;
                elif F_u[0, W_cell_U] <= 0:
                    a_W = - D;
                if F_v[0, N_cell_V] >= 0 or alternative_V >= 0:
                    a_S = -F_v[0, S_cell_V] - D;
                    a_N = -D
                elif F_v[0, N_cell_V] < 0 or alternative_V < 0:
                    a_S = - D;
                    a_N = F_v[0, N_cell_V] - D;
    
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[N, 2] = a_N
                sol_mat[P, 2] = (- F_u[0, W_cell_U] + F_v[0, N_cell_V] - F_v[0, S_cell_V]) - a_W - a_N - a_S + 2*D;
            elif mug[a_y, a_x + 1] == 0: #S wall
                sol_mat[S, 2] = 0;
                
                if F_v[0, N_cell_V] >= 0:
                    a_N = -D;
                elif F_v[0, N_cell_V] < 0:
                    a_N = F_v[0, N_cell_V] - D;
                if F_u[0, E_cell_U] >= 0 or alternative_U >= 0:
                    a_W = -F_u[0, W_cell_U] - D;
                    a_E = -D
                elif F_u[0, E_cell_U] < 0 or alternative_U < 0:
                    a_W = - D;
                    a_E = F_u[0, E_cell_U] - D;
                
                sol_mat[W, 2] = a_W;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
                sol_mat[P, 2] = (F_u[0, E_cell_U] - F_u[0, W_cell_U] + F_v[0, N_cell_V]) - a_E - a_W - a_N + 2*D;
            elif mug[a_y + 2, a_x + 1] == 0: #N wall
                sol_mat[N, 2] = 0;
                
                if F_v[0, S_cell_V] > 0:
                    a_S = - F_v[0, S_cell_V] - D;
                elif F_v[0, S_cell_V] <= 0:
                    a_S = - D;
                if F_u[0, E_cell_U] >= 0 or alternative_U >= 0:
                    a_W = -F_u[0, W_cell_U] - D;
                    a_E = -D
                elif F_u[0, E_cell_U] < 0 or alternative_U < 0:
                    a_W = - D;
                    a_E = F_u[0, E_cell_U] - D;
                
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[E, 2] = a_E;
                sol_mat[P, 2] = (F_u[0, E_cell_U] - F_u[0, W_cell_U] - F_v[0, S_cell_V] ) - a_E - a_W - a_S + 2*D;
            else: 
            #interior
                Pe_u = abs((F_u[0, E_cell_U]+F_u[0, W_cell_U])/(2*D));
                Pe_v = abs((F_v[0, N_cell_V]+F_v[0, S_cell_V])/(2*D)); #compute it if it is required for printing
                
                if ((F_u[0, E_cell_U] > 0 and F_v[0, N_cell_V] > 0) or (alternative_U >= 0 and alternative_V >= 0)) :
                    a_S = -F_v[0, S_cell_V] - D;
                    a_W = -F_u[0, W_cell_U] - D;
                    a_E = - D;
                    a_N = - D;
                    a_P = (F_u[0, E_cell_U] - F_u[0, W_cell_U] + F_v[0, N_cell_V] - F_v[0, S_cell_V] - a_E - a_N - a_W - a_S );
                 
                    sol_mat[S, 2] = a_S;
                    sol_mat[W, 2] = a_W;
                    sol_mat[P, 2] = a_P;
                    sol_mat[E, 2] = a_E;
                    sol_mat[N, 2] = a_N;
                elif ((F_u[0, E_cell_U] > 0 and F_v[0, N_cell_V] < 0) or (alternative_U >= 0 and alternative_V <= 0)):
                    a_S = - D;
                    a_W = -F_u[0, W_cell_U] - D;
                    a_E = - D;
                    a_N = F_v[0, N_cell_V] - D ;
                    a_P = (F_u[0, E_cell_U] - F_u[0, W_cell_U] + F_v[0, N_cell_V] - F_v[0, S_cell_V] - a_E - a_N - a_W - a_S );
             
                    sol_mat[S, 2] = a_S;
                    sol_mat[W, 2] = a_W;
                    sol_mat[P, 2] = a_P;
                    sol_mat[E, 2] = a_E;
                    sol_mat[N, 2] = a_N;
                elif ((F_u[0, E_cell_U] < 0 and F_v[0, N_cell_V] > 0) or (alternative_U <= 0 and alternative_V >= 0)):
                    a_S = -F_v[0, S_cell_V] - D;
                    a_W = - D;                        
                    a_E = F_u[0, E_cell_U] - D;
                    a_N = - D ;
                    a_P = (F_u[0, E_cell_U] - F_u[0, W_cell_U] + F_v[0, N_cell_V] - F_v[0, S_cell_V] - a_E - a_N - a_W - a_S );
                 
                    sol_mat[S, 2] = a_S;
                    sol_mat[W, 2] = a_W;
                    sol_mat[P, 2] = a_P;
                    sol_mat[E, 2] = a_E;
                    sol_mat[N, 2] = a_N;
                elif ((F_u[0, E_cell_U] < 0 and F_v[0, N_cell_V] < 0) or (alternative_U <= 0 and alternative_V <= 0)):
                    a_S = - D;
                    a_W = - D;
                    a_E = F_u[0, E_cell_U] - D;
                    a_N = F_v[0, N_cell_V] - D ;
                    a_P = (F_u[0, E_cell_U] - F_u[0, W_cell_U] + F_v[0, N_cell_V] - F_v[0, S_cell_V] - a_E - a_N - a_W - a_S );
                 
                    sol_mat[S, 2] = a_S;
                    sol_mat[W, 2] = a_W;
                    sol_mat[P, 2] = a_P;
                    sol_mat[E, 2] = a_E;
                    sol_mat[N, 2] = a_N;

In [ ]:
#Creation of the source term

b_source = np.zeros((1, int(sum(cells_per_row))))

for a_y in range(0, N_d):
    for a_x in range(0, N_d):
        
        num_0_left = (N_d - cells_per_row[a_y + 1])/2;
        location_cell = int(sum(cells_per_row[1 : (a_y + 1)]) + a_x - num_0_left)
            
        
        #ATTENTION THAT THE DOMAIN IS UPSIDE-DOWN
        #consider the boundary source terms
        if mug[a_y + 1, a_x + 1] == 0:
            continue
        #Boundaries
        elif mug[a_y + 1, a_x + 1] == 1:
            
            #no need to consider fluxes since the cource terms are only diffusive, i.e. heat diffusion from the boundaries
            #velocity is 0 at the wall, don't take into account of the rapid change from max velocity to 0
            #since we assume that it happens over a very thin layer (few micrometers)
            
            #condition is wall at fixed temperature
            #all the terms involved are diffusive and positive
            
            #condition is heat flux in and out
            #remove diffusivity
            
            if (mug[a_y + 1, a_x] == 0 and mug[a_y, a_x + 1] == 0): #S, W corner
                b_source[0, location_cell] = 2*D*mug_BC[a_y + 1, a_x] + 2*D*mug_BC[a_y, a_x + 1];
            elif (mug[a_y + 1, a_x + 2] == 0 and mug[a_y, a_x + 1] == 0): #S, E corner
                b_source[0, location_cell] = 2*D*mug_BC[a_y + 1, a_x + 2] + 2*D*mug_BC[a_y, a_x + 1];
            elif (mug[a_y + 1, a_x] == 0 and mug[a_y + 2, a_x + 1] == 0): #N, W corner    
                b_source[0, location_cell] = 2*D*mug_BC[a_y + 1, a_x] + 2*D*mug_BC[a_y + 2, a_x + 1];
            elif (mug[a_y + 1, a_x + 2] == 0 and mug[a_y + 2, a_x + 1] == 0): #N, E corner
                b_source[0, location_cell] = 2*D*mug_BC[a_y + 1, a_x + 2] + 2*D*mug_BC[a_y + 2, a_x + 1];
            elif mug[a_y + 1, a_x] == 0: #W wall
                b_source[0, location_cell] = 2*D*mug_BC[a_y + 1, a_x];
            elif mug[a_y + 1, a_x + 2] == 0: #E wall
                b_source[0, location_cell] = 2*D*mug_BC[a_y + 1, a_x + 2];
            elif mug[a_y, a_x + 1] == 0: #S wall
                b_source[0, location_cell] = 2*D*mug_BC[a_y, a_x + 1];
            elif mug[a_y + 2, a_x + 1] == 0: #N wall
                b_source[0, location_cell] = 2*D*mug_BC[a_y + 2, a_x + 1];
            else: 
                continue

In [ ]:
#define a temperature vector

scalar_CV = np.zeros((1,int(sum(cells_per_row))));

In [ ]:
def CGS_solver(sol_mat, b_source, guess_0):
    a, b = np.shape(b_source); #size of the system, needed for stopping the algorithm
    columns = int(b);
    
    def matrix_product_5bands(matrix, vector): #specifically for the sparse sol_mat matrix
        rows, columns = np.shape(vector);
        product = np.zeros((1, int(columns))); #list of zeros
        
        for a in range(0, columns):
            t_1 = int(a*5);
            t_2 = int(t_1 +1);
            t_3 = int(t_2 +1);
            t_4 = int(t_3 +1);
            t_5 = int(t_4 +1);
            
            S = int(sol_mat[t_1, 1]);
            W = int(sol_mat[t_2, 1]);
            P = int(sol_mat[t_3, 1]);
            E = int(sol_mat[t_4, 1]);
            N = int(sol_mat[t_5, 1]);
            
            term = sol_mat[t_1, 2]*vector[0, S] + sol_mat[t_2, 2]*vector[0, W] + sol_mat[t_3, 2]*vector[0, P] + sol_mat[t_4, 2]*vector[0, E] + sol_mat[t_5, 2]*vector[0, N];
            
            product[0, a] = term
            
        return product
    
    #initialize with 0th residual
    residual_0 = b_source - matrix_product_5bands(sol_mat, guess_0);
    
    #initialize direction vectors
    dir_vec = residual_0;
    C_dir_vec = residual_0;
    
    #initialize L-2 global residual with residual 0
    trans_residual_0 = residual_0.T;
    R2 = np.sqrt(residual_0.dot(trans_residual_0))
    
    #initialize residual for iteration
    residual = residual_0
    
    #initialize solution
    solution = guess_0
    #this will be solution at 0th iteration
    
    #set tolerance
    tolerance = 10**-8
    #reference to control the convergence
    ref = 200;
    #number of iterations
    iter_N = 0;
    
    while R2 > tolerance:
        
        #alpha^n+1, coefficient needed in solution update
        trans_residual = residual.T;
        alpha = (residual_0.dot(trans_residual))/(residual_0.dot(matrix_product_5bands(sol_mat, dir_vec).T));
        
        #vector needed to update solution and account for a-symmetric nature of the solution, G^n+1
        G_vec = C_dir_vec - alpha*(matrix_product_5bands(sol_mat, dir_vec));
        
        #update the solution
        solution = solution + alpha*(C_dir_vec + G_vec);
        
        #new residual and new global residual
        residual_n_1 = b_source - matrix_product_5bands(sol_mat, solution);
        trans_residual_n_1 = residual_n_1.T
        R2_temporary = np.sqrt(residual_n_1.dot(residual_n_1.T));
        R2 = R2_temporary[0, 0];
        
        #beta^n+1, needed in conjugate direction, search direction vector update
        beta = (residual_0.dot(trans_residual_n_1))/(residual_0.dot(trans_residual));
        
        #update conjugate direction
        C_dir_vec = residual_n_1 + beta*G_vec;
        
        #update search direction
        dir_vec = C_dir_vec + beta*(G_vec + beta*dir_vec);
        
        residual = residual_n_1
        
        print(solution[0, ref], R2, iter_N)
        
        if iter_N == columns:
            break
        
        iter_N = iter_N + 1
        
    return solution

In [ ]:
guess_0 = scalar_CV

scalar_CV = CGS_solver(sol_mat, b_source, guess_0);

In [ ]:
#PREPARE RESULT TO BE SAVED AS .xls file

scalar_CV_2 = scalar_CV.T
temperature_list = scalar_CV_2.tolist();

result = {
    "temperature": temperature_list
    
}

temp_out = pd.DataFrame(result)

In [ ]:
#PLOTTING THE RESULTS
#must pay attention that we are using a staggered arrangement
#this means that the arrangement used for the scalar CV is different from the one used for the velocity

#plotting temeperature scalar field

#must include the boundary
X_B = np.linspace(del_l/2, diam + del_l/2, N_d + 2);
Y_B = np.linspace(del_l/2, diam + del_l/2, N_d + 2);

print(X_B)

temperature_field = mug_BC

for a_y in range(0, N_d):
    for a_x in range(0, N_d):
        
        num_0_left = (N_d - cells_per_row[a_y + 1])/2;
        location_cell = int(sum(cells_per_row[1 : (a_y + 1)]) + a_x - num_0_left)
            
        
        #ATTENTION THAT THE DOMAIN IS UPSIDE-DOWN
        if mug[a_y + 1, a_x + 1] == 0:
            continue
        #Boundaries
        elif mug[a_y + 1, a_x + 1] == 1:
            temperature_field[a_y + 1, a_x + 1] = scalar_CV[0, location_cell];
    
#plot the velocity field with the velocity centred at scalar CV (needed to plot easily)

fig, axes = plt.subplots(1, 2, figsize = (10, 6));
gr_velocity_field = axes[0];
gr_temperature_field = axes[1];

gr_velocity_field.streamplot(X, Y, U, V, linewidth=1, density=1, arrowstyle='->', arrowsize=1.5); #defined at the beginning
gr_velocity_field.set_xlabel("x");
gr_velocity_field.set_ylabel("y");
gr_velocity_field.set_title("velocity field (U, V)");

temp_distr = gr_temperature_field.contourf(X_B, Y_B, temperature_field);
plt.colorbar(temp_distr, ax = gr_temperature_field);
gr_temperature_field.set_xlabel("x");
gr_temperature_field.set_ylabel("y");
gr_temperature_field.set_title("temperature field");

plt.savefig("mug_vortex_w=0.05_T_p=1_T_d=11diam5tol8.jpeg");

temperature_final = pd.DataFrame(temperature_field);
temperature_final.to_excel("T_final_distr_circular_vortex_w=0.05_T_p=1_T_d=11diam5tol8.xlsx")

In [ ]:
#using GS method, diameter 10, size cell = 0.1
#10000 iterations 23 mins good convergence
#5000 iteration 11.30 (this looks like the minimum number)

#will try 40000 to check if the solution is actually fully converged or not
#46 mins of work